In [3]:
from mesa import Agent, Model 
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
import numpy as np
import pandas as pd
import time
import datetime
import random


In [4]:
class searchAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.random.seed(SEED)

    def move(self):
        neighbors = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = self.random.choice(neighbors)

        # determinar si es comida o no, si es agente no pasar por esa celda
        # si es comida, informar a los demás agentes de la pos de la comida
        if not self.model.grid.is_cell_empty(new_position):
            return
        self.model.grid.move_agent(self, new_position)

    #informa a los demás agentes de la pos de la comida
    def inform(self):
        return
        

    def step(self):
        if self.model.is_dirty(self.pos):
            self.model.is_clean(self.pos)
            self.inform()
        else:
            self.move()

In [3]:
class pickupAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.random.seed(SEED)
        self.carrying = False

    def move(self):
        neighbors = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = self.random.choice(neighbors)

        # determinar si es comida o no, si es agente no pasar por esa celda
        # si es comida, llamar a la funcion pickupFood
        if not self.model.grid.is_cell_empty(new_position):
            return
        self.model.grid.move_agent(self, new_position)

    #informar de la comida a los demas agentes??
    def inform(self):
        return

    def pickupFood(self):
        return
    


    def step(self):
        if self.model.is_dirty(self.pos):
            self.model.is_clean(self.pos)
            self.cleaning_step += 1
        else:
            self.move()

In [4]:
class CleaningModel(Model):
    def __init__(self, width, height, num_agents, dirty_cells, max_steps):
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.running = True
        self.steps = 0
        self.max_steps = max_steps
        self.total_cells = width * height
        self.dirt = np.zeros((width, height))

        total_dirty_cells = int(dirty_cells * self.total_cells)

        all_cells = [(x, y) for x in range(width) for y in range(height)]

        np.random.shuffle(all_cells)
        dirty_cells = all_cells[:total_dirty_cells]

        for cell in dirty_cells:
            self.dirt[cell] = 1

        for i in range(self.num_agents):
            agent = CleaningAgent(i, self)
            self.grid.place_agent(agent, (1, 1))
            self.schedule.add(agent)

    def is_carrying(self, pos):
        return 
    
    def step(self):
        self.schedule.step()
        if self.steps >= self.max_steps:
            self.running = False
        self.steps += 1


In [5]:
def run_pickup_simulation(width, height, num_agents, dirty_cells, max_steps):
    model = CleaningModel(width, height, num_agents, dirty_cells, max_steps)
    for step in range(max_steps):
        model.step()
        if np.all(model.dirt == 0):
            return (
                step,
                (1 -np.mean(model.dirt)) * 100
            )

    return (
        max_steps,
        (1 - np.mean(model.dirt)) * 100
    )

In [6]:
def generate_random_food_cell():
    return random.randint(2, 5)

TOTAL_FOOD_CELLS = generate_random_food_cell()/10

def generate_random_agent():
    return random.randint(2, 3)

TOTAL_SEARCH_AGENTS = generate_random_agent()
TOTAL_PICKUP_AGENTS = 5 - TOTAL_SEARCH_AGENTS

In [7]:


WIDTH = 20
HEIGHT = 20
NUM_AGENTS = TOTAL_PICKUP_AGENTS + TOTAL_SEARCH_AGENTS
DIRTY_CELLS = TOTAL_FOOD_CELLS  # Porcentaje de celdas sucias
TOTAL_FOOD = 47
SEED = 12345
MAX_GENERATIONS = 40000

start_time = time.time()
result = run_pickup_simulation(WIDTH, HEIGHT, NUM_AGENTS, DIRTY_CELLS, MAX_GENERATIONS)
total_time = time.time() - start_time


# Imprimimos el tiempo que le tomó correr al modelo y el número de pasos al final
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=total_time)))
print("Steps:" + str(result[0]))
print ("Celdas limpias: " + str(result [1]) + "%")

NameError: name 'CleaningAgent' is not defined